<div align="center">
  <h1>65050437 นพกร แก้วสลับนิล</h1>
</div>

# Assignment 2 - Text classification

In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:

1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming)

In this homework, you are asked to do the following tasks:

1. Data Cleaning
2. Preprocessing data for keras
3. Build and evaluate a model for "action" classification
4. Build and evaluate a model for "object" classification
5. Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go

Note: we have removed phone numbers from the dataset for privacy purposes.

Please submit

1. **a colab worksheet** (a link to your worksheet or .ipynb file)
   **NLP google classroom assignment 2 by Wed 21st Feb 11.59 pm. Late submission will deduct 1% per day.**


In [1]:
!wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv

--2024-02-21 13:21:54--  https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/37u83g55p19kvrl/clean-phone-data-for-students.csv [following]
--2024-02-21 13:21:54--  https://www.dropbox.com/s/raw/37u83g55p19kvrl/clean-phone-data-for-students.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc22193260c2122dc345005980d4.dl.dropboxusercontent.com/cd/0/inline/CNutj328ng2ww8yQ1p-i7Wni0lNrdTno5aGTYxpvDkE8AnOnRehXLNVU8ENPF80D4Sdj33F0MVxEM7vR04-qYK0vB59Ic7PHvaANePpfToyV6zIdTzPLcOlreIbu1mVdDFE/file# [following]
--2024-02-21 13:21:54--  https://uc22193260c2122dc345005980d4.dl.dropboxusercontent.com/cd/0/inline/CNutj328ng2ww8yQ1p-i7Wni0lNrdTno5aGTYxpvDkE8AnOnRehXLNVU8ENP

## Import Libs


In [2]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display

import matplotlib.pyplot as plt

## Loading data

First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.


In [36]:
data_df = pd.read_csv("clean-phone-data-for-students.csv")
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

## #TODO 1:

You will have to remove unwanted label duplications as well as duplications in text inputs.
Also, you will have to trim out unwanted whitespaces from the text inputs.
This shouldn't be too hard, as you have already seen it in the demo.


In [37]:
data_df["Object"] = data_df["Object"].str.lower().str.strip()
data_df["Action"] = data_df["Action"].str.lower().str.strip()
# data_df = data_df.groupby("Sentence Utterance").first().reset_index()
data_df = data_df.drop_duplicates(subset=["Sentence Utterance"])

display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,13389,13389,13389
unique,13389,8,26
top,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,service
freq,1,8658,2111


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

array(['enquire', 'report', 'cancel', 'buy', 'activate', 'request',
       'garbage', 'change'], dtype=object)

In [38]:
action_to_idx = {action: idx for idx, action in enumerate(data_df.Action.unique())}
object_to_idx = {object: idx for idx, object in enumerate(data_df.Object.unique())}

data_df["Action_Label"] = data_df["Action"].map(action_to_idx)
data_df["Object_Label"] = data_df["Object"].map(object_to_idx)

display(data_df.head())

,Sentence Utterance,Action,Object,Action_Label,Object_Label
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment,0,0
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package,0,1
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend,1,2
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet,0,3
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues,1,4


## #TODO 2: Preprocessing data for Keras

You will be using Tensorflow 2 keras in this assignment. Please show us how you prepare your data for keras.
Don't forget to split data into train and test sets (+ validation set if you want)


In [6]:
%%capture
!pip install tensorflow_text -q

In [40]:
#### TO DO 2: Preprocessing data for Keras ###

import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [41]:
utterances = data_df["Sentence Utterance"]
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

result = embed(utterances)

In [43]:
data_df["feature_USE"] = pd.DataFrame(((x,) for x in result), columns=["lists"])

In [55]:
data_df["feature_word2vec"] = pd.Series(result.numpy().tolist())

In [56]:
display(data_df.head())

,Sentence Utterance,Action,Object,Action_Label,Object_Label,feature_USE,feature_word2vec
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment,0,0,"(tf.Tensor(-0.05400916, shape=(), dtype=float3...","[-0.05400915816426277, 0.062232837080955505, -..."
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package,0,1,"(tf.Tensor(-0.016392507, shape=(), dtype=float...","[-0.016392506659030914, 0.046880293637514114, ..."
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend,1,2,"(tf.Tensor(-0.06142924, shape=(), dtype=float3...","[-0.06142923980951309, 0.01272156834602356, -0..."
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet,0,3,"(tf.Tensor(-0.0103771975, shape=(), dtype=floa...","[-0.010377197526395321, 0.05215858668088913, -..."
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues,1,4,"(tf.Tensor(0.037256993, shape=(), dtype=float3...","[0.03725699335336685, -0.042381469160318375, 0..."


In [46]:
action_encoder = LabelEncoder()
action_encoded = action_encoder.fit_transform(data_df["Action_Label"])
action_encoded = to_categorical(action_encoded)

object_encoder = LabelEncoder()
object_encoded = object_encoder.fit_transform(data_df["Object_Label"])
object_encoded = to_categorical(object_encoded)

In [57]:
X_train, X_test, y_action_train, y_action_test, y_object_train, y_object_test = (
    train_test_split(
        data_df["feature_USE"], action_encoded, object_encoded, test_size=0.2
    )
)

In [58]:
train_features = np.array([features.numpy() for features in X_train])
test_features = np.array([features.numpy() for features in X_test])

## #TODO 3: Build and evaluate a model for "action" classification.

Please include the classification report from the test set you separate in the second step.


In [59]:
# TODO 3: Build and evaluate a model for "action" classification

classes_action = y_action_train.shape[1]

In [60]:
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout

In [87]:
action_model = Sequential(name="action_model")
action_model.add(InputLayer(input_shape=(512,)))
action_model.add(Dense(1000, activation="relu"))
action_model.add(Dropout(0.5))
action_model.add(Dense(1000, activation="relu"))
action_model.add(Dropout(0.5))
action_model.add(Dense(1000, activation="relu"))
action_model.add(Dropout(0.5))
action_model.add(Dense(classes_action, activation="softmax"))
action_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
action_model.summary()

Model: "action_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 1000)              513000    
                                                                 
 dropout_10 (Dropout)        (None, 1000)              0         
                                                                 
 dense_19 (Dense)            (None, 1000)              1001000   
                                                                 
 dropout_11 (Dropout)        (None, 1000)              0         
                                                                 
 dense_20 (Dense)            (None, 1000)              1001000   
                                                                 
 dropout_12 (Dropout)        (None, 1000)              0         
                                                                 
 dense_21 (Dense)            (None, 8)                

In [88]:
action_model.fit(train_features, y_action_train, epochs=10, batch_size=128)
action_model.save("action_model.h5", save_format="h5")

action_model.evaluate(test_features, y_action_test)

Epoch 1/10
84/84 [==============================] - 2s 4ms/step - loss: 0.8540 - accuracy: 0.7262
Epoch 2/10
84/84 [==============================] - 0s 4ms/step - loss: 0.4875 - accuracy: 0.8329
Epoch 3/10
84/84 [==============================] - 0s 4ms/step - loss: 0.4204 - accuracy: 0.8584
Epoch 4/10
84/84 [==============================] - 0s 4ms/step - loss: 0.3743 - accuracy: 0.8699
Epoch 5/10
84/84 [==============================] - 0s 4ms/step - loss: 0.3281 - accuracy: 0.8841
Epoch 6/10
84/84 [==============================] - 0s 4ms/step - loss: 0.2898 - accuracy: 0.8967
Epoch 7/10
84/84 [==============================] - 0s 4ms/step - loss: 0.2679 - accuracy: 0.9057
Epoch 8/10
84/84 [==============================] - 0s 4ms/step - loss: 0.2417 - accuracy: 0.9131
Epoch 9/10
84/84 [==============================] - 0s 4ms/step - loss: 0.2193 - accuracy: 0.9220
Epoch 10/10
84/84 [==============================] - 0s 4ms/step - loss: 0.1942 - accuracy: 0.9298


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


84/84 [==============================] - 0s 2ms/step - loss: 0.5340 - accuracy: 0.8465


[0.5339936017990112, 0.8465272784233093]

In [80]:
from sklearn.metrics import classification_report

In [89]:
predictions = action_model.predict(test_features)

print(classification_report(y_action_test.argmax(axis=1), predictions.argmax(axis=1)))

84/84 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      1761
           1       0.76      0.69      0.72       270
           2       0.87      0.95      0.91       222
           3       0.69      0.73      0.71       152
           4       0.68      0.66      0.67       105
           5       0.63      0.46      0.53        52
           6       0.00      0.00      0.00        10
           7       0.67      0.92      0.78       106

    accuracy                           0.85      2678
   macro avg       0.65      0.66      0.65      2678
weighted avg       0.84      0.85      0.84      2678



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## #TODO 4: Build and evaluate a model for "object" classification.

Please include the classification report from the test set you separate in the second step.


In [90]:
# TODO 4: Build and evaluate a model for "object" classification

classes_object = y_object_train.shape[1]

object_model = Sequential(name="object_model")
object_model.add(InputLayer(input_shape=(512,)))
object_model.add(Dense(1000, activation="relu"))
object_model.add(Dropout(0.5))
object_model.add(Dense(1000, activation="relu"))
object_model.add(Dropout(0.5))
object_model.add(Dense(1000, activation="relu"))
object_model.add(Dropout(0.5))
object_model.add(Dense(classes_object, activation="softmax"))
object_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
object_model.summary()

Model: "object_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 1000)              513000    
                                                                 
 dropout_13 (Dropout)        (None, 1000)              0         
                                                                 
 dense_23 (Dense)            (None, 1000)              1001000   
                                                                 
 dropout_14 (Dropout)        (None, 1000)              0         
                                                                 
 dense_24 (Dense)            (None, 1000)              1001000   
                                                                 
 dropout_15 (Dropout)        (None, 1000)              0         
                                                                 
 dense_25 (Dense)            (None, 26)               

In [91]:
object_model.fit(train_features, y_object_train, epochs=10, batch_size=128)
object_model.save("object_model.h5", save_format="h5")

object_model.evaluate(test_features, y_object_test)

Epoch 1/10
84/84 [==============================] - 3s 6ms/step - loss: 1.8910 - accuracy: 0.4468
Epoch 2/10
84/84 [==============================] - 1s 6ms/step - loss: 1.2252 - accuracy: 0.6220
Epoch 3/10
84/84 [==============================] - 1s 6ms/step - loss: 1.0155 - accuracy: 0.6859
Epoch 4/10
84/84 [==============================] - 0s 5ms/step - loss: 0.9194 - accuracy: 0.7143
Epoch 5/10
84/84 [==============================] - 1s 7ms/step - loss: 0.8376 - accuracy: 0.7421
Epoch 6/10
84/84 [==============================] - 1s 8ms/step - loss: 0.7526 - accuracy: 0.7629
Epoch 7/10
84/84 [==============================] - 1s 9ms/step - loss: 0.6963 - accuracy: 0.7790
Epoch 8/10
84/84 [==============================] - 1s 9ms/step - loss: 0.6316 - accuracy: 0.7974
Epoch 9/10
84/84 [==============================] - 1s 8ms/step - loss: 0.5890 - accuracy: 0.8083
Epoch 10/10
84/84 [==============================] - 1s 8ms/step - loss: 0.5405 - accuracy: 0.8249


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


84/84 [==============================] - 1s 6ms/step - loss: 0.9360 - accuracy: 0.7237


[0.9360136389732361, 0.7236743569374084]

In [92]:
predictions = object_model.predict(test_features)

print(classification_report(y_object_test.argmax(axis=1), predictions.argmax(axis=1)))

84/84 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.64      0.62      0.63       137
           1       0.71      0.73      0.72       368
           2       0.69      0.79      0.74       131
           3       0.79      0.82      0.80       360
           4       0.66      0.54      0.60       112
           5       0.74      0.76      0.75       425
           6       0.68      0.27      0.39        48
           7       0.74      0.83      0.78       299
           8       0.50      0.36      0.42        59
           9       0.74      0.70      0.72       113
          10       0.84      0.72      0.78        29
          11       0.65      0.72      0.69       232
          12       0.71      0.61      0.66        66
          13       0.67      0.50      0.57         4
          14       0.87      0.80      0.84        51
          15       0.93      0.63      0.75        41
          16       0.53     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## #TODO 5: Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go

This can be tricky if you are unfamiliar with the Keras functional API. PLEASE READ these webpages(https://www.tensorflow.org/guide/keras/functional, https://keras.io/getting-started/functional-api-guide/) before you start this task.

Your model will have two separate output layers, one for the action classification task and another for the object classification task.

This is a rough sketch of what your model might look like:
![image](https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW5/multitask_sketch.png)

Hint: You can search how to do it with "Keras Single Input Multiple Outputs. " One of the methods is to concatenate [Output1, Output2] when building a shared multi-tasked model.


In [86]:
from keras.models import Model
from keras.layers import Input

In [98]:
# TODO 5: Build and evaluate a model for multi-task classification
classes_action = y_action_train.shape[1]
action_output = Dense(classes_action, activation="softmax", name="action_output")(
    action_model.output
)

classes_object = y_object_train.shape[1]
object_output = Dense(classes_object, activation="softmax", name="object_output")(
    object_model.output
)

model = Model(
    inputs=[action_model.input, object_model.input],
    outputs=[action_output, object_output],
)
model.compile(
    optimizer="adam",
    loss={
        "action_output": "categorical_crossentropy",
        "object_output": "categorical_crossentropy",
    },
    metrics={"action_output": "accuracy", "object_output": "accuracy"},
)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 input_9 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 dense_18 (Dense)            (None, 1000)                 513000    ['input_8[0][0]']             
                                                                                                  
 dense_22 (Dense)            (None, 1000)                 513000    ['input_9[0][0]']             
                                                                                            

In [99]:
model.fit(
    [train_features, train_features],
    [y_action_train, y_object_train],
    epochs=10,
    batch_size=128,
)
model.save("multitask_model.h5", save_format="h5")

model.evaluate([test_features, test_features], [y_action_test, y_object_test])

Epoch 1/10
84/84 [==============================] - 5s 10ms/step - loss: 4.7683 - action_output_loss: 1.7092 - object_output_loss: 3.0590 - action_output_accuracy: 0.6389 - object_output_accuracy: 0.1493
Epoch 2/10
84/84 [==============================] - 1s 8ms/step - loss: 4.3651 - action_output_loss: 1.5003 - object_output_loss: 2.8648 - action_output_accuracy: 0.6693 - object_output_accuracy: 0.2163
Epoch 3/10
84/84 [==============================] - 1s 7ms/step - loss: 4.0722 - action_output_loss: 1.3448 - object_output_loss: 2.7274 - action_output_accuracy: 0.6668 - object_output_accuracy: 0.2904
Epoch 4/10
84/84 [==============================] - 1s 7ms/step - loss: 3.8279 - action_output_loss: 1.2230 - object_output_loss: 2.6050 - action_output_accuracy: 0.6731 - object_output_accuracy: 0.3128
Epoch 5/10
84/84 [==============================] - 1s 7ms/step - loss: 3.6294 - action_output_loss: 1.1291 - object_output_loss: 2.5003 - action_output_accuracy: 0.7433 - object_output_a

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


84/84 [==============================] - 1s 4ms/step - loss: 3.0576 - action_output_loss: 0.8927 - object_output_loss: 2.1649 - action_output_accuracy: 0.7636 - object_output_accuracy: 0.3820


[3.0575954914093018,
 0.8927391171455383,
 2.164856195449829,
 0.7636295557022095,
 0.38200148940086365]

In [100]:
predictions = model.predict([test_features, test_features])

print(
    classification_report(y_action_test.argmax(axis=1), predictions[0].argmax(axis=1))
)
print(
    classification_report(y_object_test.argmax(axis=1), predictions[1].argmax(axis=1))
)

84/84 [==============================] - 0s 4ms/step
              precision    recall  f1-score   support

           0       0.87      0.86      0.87      1761
           1       0.56      0.87      0.68       270
           2       0.53      0.91      0.67       222
           3       0.00      0.00      0.00       152
           4       0.64      0.65      0.64       105
           5       0.85      0.42      0.56        52
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00       106

    accuracy                           0.76      2678
   macro avg       0.43      0.46      0.43      2678
weighted avg       0.71      0.76      0.73      2678

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       137
           1       0.49      0.08      0.14       368
           2       0.00      0.00      0.00       131
           3       0.36      0.77      0.49       360
           4       0.56  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif